# Getting the audio features

In this notebook we are going to get the audio features of all the songs in an album. A list explaining Spotify's audio metrics can be found [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/)

In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
from json import load

# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

## Getting your credentials and logging in

In [2]:
scope = 'user-library-read'

with open('credentials.txt') as fd:
    creds = load(fd)

In [3]:
def get_token(scope, **kwargs):
    try:
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    except:
        os.remove(f".cache-{kwargs['username']}")
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    return token

In [4]:
token = get_token(scope, **creds)
sp = spotipy.Spotify(auth=token)

## Getting the album tracks info

In order to get the audio features of every track in an album we need the ids of each track.

In [5]:
album = sp.album_tracks('spotify:album:6trNtQUgC8cgbWcqoMYkOR')
album.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
# From the keys we can see that the track names and ids are one level in the 'items' list
album['items'][0].keys()

dict_keys(['artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

In [7]:
song_name, song_id = album['items'][0]['name'], album['items'][0]['id']
song_name, song_id

('Paranoid', '3fpVWegR6YOS1Yk5HSMYIq')

We can now define a function that will return the names and ids of all songs in an album.

In [8]:
def get_album_track_names_ids(album_id):
    """
    This function returns two lists where the first has the names 
    of every track and the second are the associated track ids. 
    """
    album = sp.album_tracks(album_id)
    names, ids = [], []
    
    for track in album['items']:
        names.append(track['name'])
        ids.append(track['id'])
    
    return names, ids

In [9]:
names, ids = get_album_track_names_ids('spotify:album:6trNtQUgC8cgbWcqoMYkOR')

In [10]:
# Now that we have the ids we can get a list of all the audio_features of each song.
sp.audio_features(tracks=ids)

[{'danceability': 0.61,
  'energy': 0.554,
  'key': 0,
  'loudness': -5.501,
  'mode': 1,
  'speechiness': 0.137,
  'acousticness': 0.0379,
  'instrumentalness': 3.84e-05,
  'liveness': 0.166,
  'valence': 0.244,
  'tempo': 159.85,
  'type': 'audio_features',
  'id': '3fpVWegR6YOS1Yk5HSMYIq',
  'uri': 'spotify:track:3fpVWegR6YOS1Yk5HSMYIq',
  'track_href': 'https://api.spotify.com/v1/tracks/3fpVWegR6YOS1Yk5HSMYIq',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3fpVWegR6YOS1Yk5HSMYIq',
  'duration_ms': 221787,
  'time_signature': 4},
 {'danceability': 0.669,
  'energy': 0.675,
  'key': 1,
  'loudness': -2.869,
  'mode': 1,
  'speechiness': 0.0337,
  'acousticness': 0.337,
  'instrumentalness': 1.5e-05,
  'liveness': 0.109,
  'valence': 0.262,
  'tempo': 160.911,
  'type': 'audio_features',
  'id': '5VuxWXbt7XENQCtE9TzpTv',
  'uri': 'spotify:track:5VuxWXbt7XENQCtE9TzpTv',
  'track_href': 'https://api.spotify.com/v1/tracks/5VuxWXbt7XENQCtE9TzpTv',
  'analysis_url': 'https:/

Finally We can make a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) to visualize the audio_features nicely

In [11]:
def get_album_DataFrame(album_id):
    names, ids = get_album_track_names_ids(album_id)
    audio_features = sp.audio_features(tracks=ids)
    df = pd.DataFrame(audio_features)
    df.insert(loc=0, column='track_name', value=names)
    return df

In [12]:
df = get_album_DataFrame('spotify:album:6trNtQUgC8cgbWcqoMYkOR')
df

,track_name,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,Paranoid,0.0379,https://api.spotify.com/v1/audio-analysis/3fpV...,0.610,221787,0.554,3fpVWegR6YOS1Yk5HSMYIq,0.000038,0,0.1660,-5.501,1,0.1370,159.850,4,https://api.spotify.com/v1/tracks/3fpVWegR6YOS...,audio_features,spotify:track:3fpVWegR6YOS1Yk5HSMYIq,0.244
1,Spoil My Night (feat. Swae Lee),0.3370,https://api.spotify.com/v1/audio-analysis/5Vux...,0.669,194560,0.675,5VuxWXbt7XENQCtE9TzpTv,0.000015,1,0.1090,-2.869,1,0.0337,160.911,4,https://api.spotify.com/v1/tracks/5VuxWXbt7XEN...,audio_features,spotify:track:5VuxWXbt7XENQCtE9TzpTv,0.262
2,Rich & Sad,0.1860,https://api.spotify.com/v1/audio-analysis/2VdT...,0.601,206453,0.590,2VdT56BGpdqNHUgOe1j5vc,0.000000,2,0.0942,-5.001,1,0.0569,151.952,4,https://api.spotify.com/v1/tracks/2VdT56BGpdqN...,audio_features,spotify:track:2VdT56BGpdqNHUgOe1j5vc,0.274
3,Zack And Codeine,0.1430,https://api.spotify.com/v1/audio-analysis/5lZZ...,0.742,204360,0.840,5lZZmityu9TAjNvSY6GLhR,0.000000,5,0.1330,-5.198,0,0.0471,138.032,4,https://api.spotify.com/v1/tracks/5lZZmityu9TA...,audio_features,spotify:track:5lZZmityu9TAjNvSY6GLhR,0.287
4,Takin' Shots,0.5970,https://api.spotify.com/v1/audio-analysis/5ffn...,0.767,216747,0.625,5ffntNJnJOeFv7b7DCd0Bw,0.000000,5,0.4090,-6.193,1,0.0836,120.092,4,https://api.spotify.com/v1/tracks/5ffntNJnJOeF...,audio_features,spotify:track:5ffntNJnJOeFv7b7DCd0Bw,0.577
5,rockstar (feat. 21 Savage),0.1170,https://api.spotify.com/v1/audio-analysis/0e7i...,0.587,218147,0.535,0e7ipj03S05BNilyu5bRzt,0.000066,5,0.1310,-6.090,0,0.0898,159.847,4,https://api.spotify.com/v1/tracks/0e7ipj03S05B...,audio_features,spotify:track:0e7ipj03S05BNilyu5bRzt,0.140
6,Over Now,0.1310,https://api.spotify.com/v1/audio-analysis/09IO...,0.559,246987,0.761,09IOPhEh1OMe0HD9b36FJk,0.000002,4,0.1430,-4.041,0,0.0877,163.898,4,https://api.spotify.com/v1/tracks/09IOPhEh1OMe...,audio_features,spotify:track:09IOPhEh1OMe0HD9b36FJk,0.240
7,Psycho (feat. Ty Dolla $ign),0.5800,https://api.spotify.com/v1/audio-analysis/3swc...,0.739,221440,0.559,3swc6WTsr7rl9DqQKQA55C,0.000000,8,0.1120,-8.011,1,0.1170,140.124,4,https://api.spotify.com/v1/tracks/3swc6WTsr7rl...,audio_features,spotify:track:3swc6WTsr7rl9DqQKQA55C,0.439
8,Better Now,0.3540,https://api.spotify.com/v1/audio-analysis/7dt6...,0.680,231267,0.563,7dt6x5M1jzdTEt8oCbisTK,0.000000,10,0.1360,-5.843,1,0.0454,145.028,4,https://api.spotify.com/v1/tracks/7dt6x5M1jzdT...,audio_features,spotify:track:7dt6x5M1jzdTEt8oCbisTK,0.374
9,Ball For Me (feat. Nicki Minaj),0.4930,https://api.spotify.com/v1/audio-analysis/6Iai...,0.792,206267,0.560,6IaieqiCVvsNvEt6Y7yOFa,0.000000,9,0.0849,-4.125,1,0.2250,134.044,4,https://api.spotify.com/v1/tracks/6IaieqiCVvsN...,audio_features,spotify:track:6IaieqiCVvsNvEt6Y7yOFa,0.505
